## 比赛介绍
量子电路合成是量子计算中十分重要的问题，对于实现量子计算机有指导意义。本次比赛以量子电路合成为题，旨在让参赛者了解并掌握如何使用给定量子门集合来高效的近似合成目标量子门，加深对量子计算的认识。  
  
详细介绍：[https://aistudio.baidu.com/aistudio/competition/detail/70](https://aistudio.baidu.com/aistudio/competition/detail/70)

## 准备环境
- 安装量桨
- 解压试题

In [ ]:
!pip install paddle-quantum

In [ ]:
!unzip -oq /home/aistudio/data/data71784/飞桨常规赛：量子电路合成.zip -d ~/data/
!mv ~/data/飞桨常规赛：量子电路合成 ~/data/target

## 开始答题

### 解题思路  
前三题给定电路结构，使用量桨搭建对应电路，用飞桨求解即可  
后三题需要猜测出电路结构，再求解参数

In [ ]:
#导入所需包
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import gate_fidelity

### 基于量桨的量子电路类

In [ ]:
class MyCircuit(UAnsatz):

    def gate_fidelity(self, target):
        a = self.U
        b = target
        a = paddle.cast(a, 'float64')
        a.stop_gradient = False
        b = paddle.cast(b, 'float64')
        b.stop_gradient = False
        c = paddle.fluid.layers.matmul(a, b, transpose_y=True)
        trace = paddle.trace(c)
        return paddle.abs(trace) / 2**self.n
    
    @property
    def complexity(self):
        c = 0
        for g, _, _ in self._UAnsatz__history:
            if g == 'u':
                c += 1
            elif g == 'CNOT':
                c += 8
        return c
    
    def output(self, path):
        history = []
        for g, w, p in self._UAnsatz__history:
            if g == 'u':
                theta = p[0]
                if hasattr(p[0], 'numpy'):
                    theta = p[0].numpy()[0]                    
                history.append('R %d %f\n' % (w[0], np.mod(theta, 2 * np.pi)))
            elif g == 'CNOT':
                history.append('C %d %d\n' % tuple(w))
            else:
                raise OSError('未知的门`%s`' % g)
        with open(path, 'w') as f:
                for i in history:
                    f.write(i)

### 基于飞桨的自动优化器

In [ ]:
class Net(paddle.nn.Layer):
    def __init__(self, n, p, dtype="float64"):
        super(Net, self).__init__()
        self.n = n
        self.theta = self.create_parameter(shape=[p], 
                                           default_initializer=paddle.nn.initializer.Uniform(low=-np.pi/2, high=np.pi/2),
                                           dtype=dtype, is_bias=False)
    def forward(self):
        cir = circuit(self.theta)
        loss = 1 - cir.gate_fidelity(target_tensor)
        return loss, cir

In [ ]:
class Optimizer:
    def __init__(self, net, learning_rate):
        scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=LR, T_max=100, eta_min=1e-3, last_epoch=99, verbose=False)
        opt = paddle.optimizer.Adam(learning_rate=scheduler, parameters=net.parameters())
        self.net = net
        self.scheduler = scheduler
        self.opt = opt

    def train(self, max_iters=500, log_iter=50, threshold=1e-5, display=True):
        last = 1
        for itr in range(1, max_iters + 1):
            loss, cir = self.net()
            # 计算梯度并优化
            loss.backward()
            self.opt.minimize(loss)
            self.opt.clear_grad()
            self.scheduler.step()
            if itr % log_iter == 0:
                l = loss.numpy()[0]
                if display:
                    print("iter:", "%3d" % itr, "  loss:", "%.4f" % l, "%.2e" % (l - last))
                if abs(l - last) < threshold:
                    break
                last = l
        return last


### 第一题 单量子比特门近似

In [ ]:
n = 1
target = 1 / 2**0.5 * np.array([[1, -1], [1, 1]])
target_tensor = paddle.to_tensor(target)
LR = 0.1
ITR = 1000

def circuit(theta):
    # 初始化 n 个量子比特的量子电路
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    return cir

In [ ]:
net = Net(n, 1)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_1_Answer.txt')

### 第二题 双量子比特门分解

In [ ]:
n = 2
target = np.loadtxt('/home/aistudio/data/target/Question_2_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.05
ITR = 1000

def circuit(theta):
    # 初始化 n 个量子比特的量子电路
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    cir.ry(theta[1], 1)
    cir.cnot([0, 1])
    cir.ry(theta[2], 0)
    cir.ry(theta[3], 1)
    return cir

In [ ]:
net = Net(n, 4)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_2_Answer.txt')

### 第三题 三量子比特门分解

In [ ]:
n = 3
target = np.loadtxt('/home/aistudio/data/target/Question_3_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.02
ITR = 1000

def circuit(theta):
    # 初始化 n 个量子比特的量子电路
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    cir.ry(theta[1], 1)
    cir.ry(theta[2], 2)
    cir.cnot([0, 1])
    cir.cnot([1, 2])
    cir.ry(theta[3], 0)
    cir.ry(theta[4], 2)
    return cir

In [ ]:
net = Net(n, 5)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_3_Answer.txt')

### 可变量子电路类
为了便于尝试不同的电路结构，定义可变量子电路类

In [ ]:
class VariableCircuit(MyCircuit):
    def __init__(self, n, theta, controls=[]):
        super().__init__(n)
        self.controls = controls
        self.init(theta)

    def init(self, theta):
        offset = 0
        for i in range(self.n):
            self.ry(theta[offset+i], i)
        offset += self.n

        for a, b in self.controls:
            self.cnot([a, b])
            self.ry(theta[offset+0], a)
            self.ry(theta[offset+1], b)
            offset += 2
        self.params_size = offset

### 第四题 三量子比特门无结构分解
这题可以使用暴力搜索找到最优结构  
最后手动将结果里参数为2pi整数倍的旋转门删掉

In [ ]:
n = 3
target = np.loadtxt('/home/aistudio/data/target/Question_4_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 1
ITR = 500

In [ ]:
def circuit(theta):
    conts = [(1,2), (1,2), (0,1), (0,2), (0,1)] #(0.9478)
    cir = VariableCircuit(n, theta, conts)
    return cir

net = Net(n, 15)
cir = circuit(net.theta)
print(cir.complexity)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-10)

In [ ]:
theta_opt = net.theta.numpy()
theta_opt = np.mod(theta_opt, 2 * np.pi)
print("优化后的参数 theta:\n", theta_opt / np.pi)

cir = circuit(net.theta)
cir.output('Question_4_Answer.txt')

### 第五题 四量子比特门无结构分解
这里使用的为之前一个版本参数，只在代码结构上进行了简化

In [ ]:
n = 4
target = np.loadtxt('/home/aistudio/data/target/Question_5_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 1.0
ITR = 1000

In [ ]:
def circuit(theta):
    conts = [(1, 2)]
    for i in range(7):
        for j in range(i%2, n, 2):
            conts.append((j%n, (j+1)%n))
    conts.append((3, 0))
    cir = VariableCircuit(n, theta, conts)
    return cir

In [26]:
while 1:
    net = Net(n, 36)
    opt = Optimizer(net, LR)
    l = opt.train(ITR)
    if l < 0.25:
        break

In [27]:
# 输出结果
theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_5_Answer.txt')

### 第六题 八量子比特门无结构分解
暂时没找到好的解决方案

## 提交结果

In [28]:
!zip Answer.zip Question_?_Answer.txt

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 